In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from fastcore.all import *
import pandas as pd

In [2]:
main_url = 'https://help.dyalog.com/18.2/Content/Language/Introduction/Language%20Elements.htm?tocpath=Language%20Reference%20Guide%7CSymbols%7C_____1'
content = requests.get(main_url)
content.encoding = content.apparent_encoding

main=BeautifulSoup(content.text)

symbol_tags = main.find_all('a',attrs={'href':lambda x: x.startswith('../Symbols/')})
symbols_dict = {o.text:o['href'] for o in symbol_tags}

In [3]:
base = 'https://help.dyalog.com/18.2/Content/Language'

Card1:
+ Symbol : Name + Monadic Name + Dyadic Name
+ Symbol Name : Symbol
+ Monadic Name : Symbol
+ Dyadic Name : Symbol
+ Documentation Name : Symbol (only if different)
+ Documentation Name : Documentation
+ Symbol : Documentation
+ Documentation : Symbol
+ Documentation : Documentation Name

In [86]:
cards = []
for i,sym in enumerate(L(*symbols_dict.keys())):
    doc_url = symbols_dict[sym]
    url1 = base + doc_url[2:]

    content = requests.get(url1)
    content.encoding = content.apparent_encoding
    doc_html = BeautifulSoup(content.text)

    doc_h5s = doc_html.find_all('h5')
    doc_h3 = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
    
    
    if doc_html.find('td').text.strip() == '':
        _2 = doc_h3['name']
    else: _2 = doc_html.find('td').text.strip()
    
    cards.append(pd.DataFrame([[_2,sym]]))
    
    for doc_h5 in doc_h5s:
        p = doc_h5.find_next_sibling('p')
        dy_or_mon_name = ''.join(re.findall('Dyadic|Monadic',doc_h5.text)) + ': ' + p.text.strip()
        cards.append(pd.DataFrame([[dy_or_mon_name,sym]]))
        _2  = _2 + '<br/>' + dy_or_mon_name 
        
        
        url_end = [o.find('a',attrs={'href':True}) for o in doc_h5.find_next_siblings('p') if o.find('a',attrs={'href':True}) is not None][0]['href']
#         url2 = base + p.find('a',attrs={'href':True})['href'][2:]
        url2 = base + url_end[2:]
        
        content = requests.get(url2)
        content.encoding = content.apparent_encoding
        doc_html = BeautifulSoup(content.text)
    
        try:
            desc = [o.text for o in doc_html.find('td').find_all_next('p') if o.text.strip() != '']

            documentation = '<br/>'.join(desc)
            cards.append(pd.DataFrame([[documentation,sym],
                                       [sym,documentation]]))
        except:
            pass
        try:
            doc_name = doc_html.find('td').find('a',attrs={'name':lambda o: not o.startswith('kanchor')})
            if doc_name['name'] not in _2:
                cards.append(pd.DataFrame([[doc_name['name'],sym]]))

        
            cards.append(pd.DataFrame([[doc_name['name'],documentation],
                               [documentation,doc_name['name']],
                              ]))
        except:
            print(i)
                
  

    cards.append(pd.DataFrame([[sym,_2]]))

65
72
72


In [87]:
df = pd.concat(cards)
df.to_csv('apl-docs.csv',index=False,header=False)
df.sample(15)

,0,1
1,Y may be any array. The axis specification is...,Reverse
1,Y must be a Boolean array. R is Boolean. The...,Not
0,Same,Y may be any array.<br/>The result R is the ar...
1,⊆,Classic Edition: the symbol ⊆ (Left Shoe Under...
0,Execute,Y must be a simple character scalar or vector ...
0,Up Arrow,↑
0,Dyadic: Catenate First,⍪
0,Circle,○
0,Iota Underbar,⍸
0,: Scan First,⍀
